In [144]:
import pandas as pd
from shapely.geometry import Polygon
import numpy as np
from shapely import affinity
from miscellaneous_functions import *

In [145]:
data = pd.read_json("game_levels/00000_0_0_00_0_0.xml.json")
#read the basemap file
basemap = pd.read_json("BaseMap.json")

In [146]:
#make the dataframe with obj data

id = 0
obj_id = [] #obj counter

label = []
coords = []
rotation = []
colours = []

for objs in data['features'][0]:
    obj_id.append(id)
    if len(objs['geometry'])!=0:
        coords.append(objs['geometry']['coordinates'][0])
    else: 
        coords.append(np.nan)
        
    label.append(objs['properties']['label'])
    colours.append(objs['properties']['colormap'])
    
    if 'rotation' in objs['properties']:
        rotation.append(objs['properties']['rotation'])
    else: rotation.append(np.nan)
    id += 1

In [147]:
data_df = {"obj_id":obj_id, "label":label, "coords": coords, "colours":colours, "rotation":rotation}
data_df = pd.DataFrame(data_df)

In [148]:
#round the datadf
for i in range(len(data_df['colours'])):   
    for j in range(len(data_df['colours'][i])):
        data_df['colours'][i][j]['color'] = round(data_df['colours'][i][j]['color'],4)
        data_df['colours'][i][j]['percent'] = round(data_df['colours'][i][j]['percent'],4)
        #print(data_df['colours'][i][j])
        
#rename to col and percent in basemap
for i in range(len(basemap['sprites'])):
    #print(basemap['sprites'][i]['colormap'])
    for j in range(len(basemap['sprites'][i]['colormap'])):
        #print(basemap['sprites'][i]['colormap'][j])
        basemap['sprites'][i]['colormap'][j]['color'] = basemap['sprites'][i]['colormap'][j].pop('x')
        basemap['sprites'][i]['colormap'][j]['percent'] = basemap['sprites'][i]['colormap'][j].pop('y')

#round the percent in basemap
for i in range(len(basemap['sprites'])):
    #print(basemap['sprites'][i]['colormap'])
    for j in range(len(basemap['sprites'][i]['colormap'])):
        basemap['sprites'][i]['colormap'][j]['color'] = round(basemap['sprites'][i]['colormap'][j]['color'],4)
        basemap['sprites'][i]['colormap'][j]['percent'] = round(basemap['sprites'][i]['colormap'][j]['percent'],4)
        #print(basemap['sprites'][i]['colormap'][j]['color'],basemap['sprites'][i]['colormap'][j]['percent'])


In [149]:
#convert basemap to a dataframe
label = []
colours = []
for i in range(len(basemap['sprites'])):
    label.append(basemap['sprites'][i]['type'])
    colours.append(basemap['sprites'][i]['colormap'])

basemap_df = {"obj_label":label, "colours":colours}
basemap_df = pd.DataFrame(basemap_df)

In [150]:
#adjust the label of the object 
for j in range(len(data_df)):
    for i in range(len(basemap_df)):
        if basemap_df['colours'][i]==data_df['colours'][j]:
            #data_df['label'][j] = basemap_df['obj_label'][i]
            data_df.loc[j,'label'] = basemap_df['obj_label'][i]
            break

In [151]:
#data_df
#add the sling rotation
data_df.loc[1,'rotation'] = 0

In [152]:
normalized_polygon_x = [np.nan]
normalized_polygon_y = [np.nan]
for i in range(1,len(data_df)):
    r1 = data_df['rotation'][i]*100
    p1 = Polygon(data_df['coords'][i])
    norm_polygon = create_normalized_polygon(p1,r1)
    x = norm_polygon.exterior.coords.xy[0]
    y = norm_polygon.exterior.coords.xy[1]
    normalized_polygon_x.append(x)
    normalized_polygon_y.append(y)
    #normalized_polygon.append(tuple(zip(x,y)))

In [153]:
norm_polygon.exterior.coords.xy[0]

array('d', [0.0, 0.0, 50.0, 50.0, 0.0])

In [154]:
data_df['normalized_polygon_x'] = normalized_polygon_x
data_df['normalized_polygon_y'] = normalized_polygon_y

In [155]:
split_names = data_df['label'].str.split('_')

In [156]:
polygon_name = []
for i in range(len(split_names)):
    if split_names[i][0]=='wood':
        if len(split_names[i])==3:
            polygon_name.append(combine_a_list(split_names[i][1:2]))
        else: 
            polygon_name.append(combine_a_list(split_names[i][1:3]))
    elif len(split_names[i])==1:
        polygon_name.append(split_names[i][0])
    else:
        polygon_name.append(combine_a_list(split_names[i][0:-1]))
        
data_df['polygon_name'] = polygon_name
data_df['polygon_name'] = data_df['polygon_name']+"_shape"

In [157]:
data_df

,obj_id,label,coords,colours,rotation,normalized_polygon_x,normalized_polygon_y,polygon_name
0,0,Ground,NaN,[],NaN,NaN,NaN,Ground_shape
1,1,Slingshot,"[[188, 408], [188, 331], [214, 331], [214, 408]]",[],0.000000e+00,"[0.0, 0.0, 26.0, 26.0, 0.0]","[77.0, 0.0, 0.0, 77.0, 77.0]",Slingshot_shape
2,2,pig_basic_small_1,"[[623, 345], [617, 349], [616, 355], [620, 359...","[{'color': 0, 'percent': 0.0446}, {'color': 4,...",-8.818279e-08,"[6.999998460919869, 0.9999990765520579, 0.0, 4...","[0.0, 4.000000923447999, 10.000001077355932, 1...",pig_basic_small_shape
3,3,pig_basic_medium_1,"[[469, 386], [470, 383], [468, 379], [462, 380...","[{'color': 0, 'percent': 0.0206}, {'color': 4,...",3.217243e-04,"[25.00841878079916, 26.010103167556736, 24.012...","[7.008421618489251, 4.008983606239951, 0.00786...",pig_basic_medium_shape
4,4,bird_white_1,"[[214, 335], [207, 329], [207, 324], [203, 323...","[{'color': 0, 'percent': 0.0346}, {'color': 36...",0.000000e+00,"[28.0, 21.0, 21.0, 17.0, 12.0, 9.0, 9.0, 6.0, ...","[12.0, 6.0, 1.0, 0.0, 0.0, 3.0, 5.0, 12.0, 18....",bird_white_shape
5,5,bird_red_1,"[[177, 393], [177, 392], [173, 392], [168, 397...","[{'color': 0, 'percent': 0.2689}, {'color': 36...",-1.331487e-01,"[9.270586752893564, 9.040284453895651, 5.14780...","[0.9731191350889503, 0.0, 0.9212091959917643, ...",bird_red_shape
6,6,bird_blue_1,"[[144, 391], [143, 393], [138, 398], [136, 396...","[{'color': 0, 'percent': 0.2245}, {'color': 37...",-1.257669e-01,"[6.719312512795966, 6.178799453126487, 2.38750...","[0.0, 2.1697570445390397, 8.13851511848668, 6....",bird_blue_shape
7,7,bird_yellow_1,"[[90, 385], [89, 386], [89, 387], [90, 387], [...","[{'color': 0, 'percent': 0.0259}, {'color': 36...",-8.106410e-02,"[6.79196718047254, 5.942971282913476, 6.083983...","[0.0, 1.1310198786616752, 2.1210277667720447, ...",bird_yellow_shape
8,8,bird_black_1,"[[45, 388], [46, 386], [47, 385], [38, 383], [...","[{'color': 0, 'percent': 0.1084}, {'color': 36...",-4.936467e-02,"[22.878364912637934, 23.702553543972925, 24.61...","[12.177320038748462, 10.09868753155098, 9.0163...",bird_black_shape
9,9,wood_circle_1,"[[929, 379], [915, 379], [914, 380], [908, 386...","[{'color': 19, 'percent': 0.0354}, {'color': 1...",5.551760e-09,"[22.000000164724042, 8.000000164724042, 7.0000...","[1.3565505696533364e-07, 0.0, 0.99999999031035...",circle_shape


In [158]:
data_df[['polygon_name','normalized_polygon_x','normalized_polygon_y']].to_excel("normalized_polygon.xlsx")